-

<h1><center>[[---- Bad Comment Generator using RNN ----]]</center></h1>
<h3><center>( 발표자료 : http://www.slideshare.net/shuraba1/bad-comment-generator-using-rnn )</center></h3>
<br><br>
<h3><p style="text-align:right">발표자 : 송치성(https://www.facebook.com/shuraba)</p></h3>

# Goal : 
<hr style="border-color:#ff9900"> 
## 영화 악평들로 학습하는 Language Modeling을 통해 '악평생성기'를 만들어보자!

<br><br><br><br>
# Step 1. Data Preprocessing
<hr style="border-color:#ff9900"> 

- 데이터를 RNN으로 학습하기 좋은 형태로 전처리.

In [31]:
import os
import time
import random
import itertools
import pickle
import collections

import numpy as np
import tensorflow as tf
from tensorflow.python.ops import rnn_cell
from tensorflow.python.ops import seq2seq

## 0. Load Text Data & Attach Sentence Start/End Token
- 실습에 사용할 데이터 불러옴.
- 실습데이터 : 평점 상위권 영화의 리뷰 중 악평들..

In [32]:
DATA_DIR = "./data"
INPUT_FILE = os.path.join(DATA_DIR, "bad_reviews.txt")

- 우리의 목적 : 이런 꼴이 되도록 만들자!
    - 토크나이징 하기 전에 문장(리뷰)의 시작과 끝에 각각 특정 토큰을 붙임.
    - 문장(리뷰)의 첫번째 단어와 마지막 단어는 어떤게 올지 학습이 가능.
![RNN example](img/RNN_example.png)

In [33]:
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [34]:
with open(INPUT_FILE, "r") as f:
    sentences = ['%s %s %s' % (sentence_start_token, line.strip(), sentence_end_token) for line in f.readlines()]

In [35]:
sentences[:8]

['SENTENCE_START 극장에서 돈내고 보긴아깝다 SENTENCE_END',
 'SENTENCE_START 진짜,, SENTENCE_END',
 'SENTENCE_START 훌륭하지만 비약적인 스토리 SENTENCE_END',
 'SENTENCE_START OOO야.. 아 진짜 후회했음 후회막급 ! ! 오빠테 욕먹었음... -_- SENTENCE_END',
 'SENTENCE_START 잼씀 SENTENCE_END',
 'SENTENCE_START 지 랄들 하세요 이민자들의 성공기 허울좋은 명분일뿐이고 마피아비판도 명분이고 마피아미화란 미화는 다 시키고 똥폼잡는작품 방구석에 처박혀서 이런거나보고 멋있다고 아우성는 저능아들을 위한작품 세상을 제대로알고 똑바로봐라 SENTENCE_END',
 'SENTENCE_START 1~3편보고솔직히재미없다 SENTENCE_END',
 'SENTENCE_START 진짜 심하게 지루하다 SENTENCE_END']

In [36]:
# 전체 리뷰 갯수
len(sentences)

128173

In [37]:
# 실습을 위한 슬라이싱.
sentences = sentences[:1000]

In [38]:
len(sentences)

1000

## 1. Tokenize Sentences into words
- __토큰화(Tokenize)__ : 자연어처리를 하기위하여 텍스트를 기본단위로 나누는 과정.
- 어절 단위 vs 형태소 단위
    - 본 실습에서는 __어절__단위로 토큰화. ( ex 동해물과/백두산이/마르고/닳도록 ...)
    - 어절단위 토큰화 장단점:
        - 장점: __빠르다__, 간단하다
        - 단점: __교착어에 취약__ (ex. "치킨을" != "치킨이", "봤다" != "봤었다")
- 이번 실습에서는 한 리뷰가 한 문장이라고 가정.

In [39]:
from konlpy.tag import Mecab
tagger = Mecab()
sample_review = "극장에서 돈내고 보긴 아깝다."

In [40]:
# 형태소 단위 Tokenizing
["%s/%s" % (word,pos) for word,pos in tagger.pos(sample_review)]

['극장/NNG',
 '에서/JKB',
 '돈/NNG',
 '내/VV',
 '고/EC',
 '보/VX',
 '긴/ETN+JX',
 '아깝/VA',
 '다/EF',
 './SF']

In [41]:
# 어절 단위 Tokenizing
sample_review.split()

['극장에서', '돈내고', '보긴', '아깝다.']

In [42]:
sentences_tokenized = [sent.split() for sent in sentences]
sentences_tokenized[:8]

[['SENTENCE_START', '극장에서', '돈내고', '보긴아깝다', 'SENTENCE_END'],
 ['SENTENCE_START', '진짜,,', 'SENTENCE_END'],
 ['SENTENCE_START', '훌륭하지만', '비약적인', '스토리', 'SENTENCE_END'],
 ['SENTENCE_START',
  'OOO야..',
  '아',
  '진짜',
  '후회했음',
  '후회막급',
  '!',
  '!',
  '오빠테',
  '욕먹었음...',
  '-_-',
  'SENTENCE_END'],
 ['SENTENCE_START', '잼씀', 'SENTENCE_END'],
 ['SENTENCE_START',
  '지',
  '랄들',
  '하세요',
  '이민자들의',
  '성공기',
  '허울좋은',
  '명분일뿐이고',
  '마피아비판도',
  '명분이고',
  '마피아미화란',
  '미화는',
  '다',
  '시키고',
  '똥폼잡는작품',
  '방구석에',
  '처박혀서',
  '이런거나보고',
  '멋있다고',
  '아우성는',
  '저능아들을',
  '위한작품',
  '세상을',
  '제대로알고',
  '똑바로봐라',
  'SENTENCE_END'],
 ['SENTENCE_START', '1~3편보고솔직히재미없다', 'SENTENCE_END'],
 ['SENTENCE_START', '진짜', '심하게', '지루하다', 'SENTENCE_END']]

In [43]:
# 전체 문장을 하나의 리스트로 변환.
text_tokenized = list(itertools.chain(*sentences_tokenized))
text_tokenized[:30]

['SENTENCE_START',
 '극장에서',
 '돈내고',
 '보긴아깝다',
 'SENTENCE_END',
 'SENTENCE_START',
 '진짜,,',
 'SENTENCE_END',
 'SENTENCE_START',
 '훌륭하지만',
 '비약적인',
 '스토리',
 'SENTENCE_END',
 'SENTENCE_START',
 'OOO야..',
 '아',
 '진짜',
 '후회했음',
 '후회막급',
 '!',
 '!',
 '오빠테',
 '욕먹었음...',
 '-_-',
 'SENTENCE_END',
 'SENTENCE_START',
 '잼씀',
 'SENTENCE_END',
 'SENTENCE_START',
 '지']

## 2. Word Indexing and Remove Infrequent Words
- 각 단어에 고유의 인덱스(id)를 붙임.
- 인덱스를 통한 one-hot vector 형태로 단어 표현. (cf. [Vector Representations of Words](https://www.tensorflow.org/versions/r0.8/tutorials/word2vec/index.html) )
- 인덱스는 빈도순으로 줌.
- 빈도수가 낮은 단어 제거 (기준 : 단어 사전 사이즈).

In [46]:
# 단어 개수 Threshold
vocab_size = 3000

In [47]:
# vocabulary 생성
word_freq = collections.Counter(text_tokenized)
vocab = word_freq.most_common(vocab_size - 1)

In [48]:
# 전체 단어 갯수
len(vocab) 

2999

In [49]:
# 상위 빈도 10개 
vocab[:10]

[('SENTENCE_START', 1000),
 ('SENTENCE_END', 1000),
 ('너무', 62),
 ('진짜', 55),
 ('영화', 48),
 ('정말', 40),
 ('평점이', 31),
 ('그냥', 29),
 ('영화.', 23),
 ('다', 20)]

In [50]:
# 하위 빈도 10개
vocab[-10:]

[('아이고', 1),
 ('영화는...이라는', 1),
 ('새것이었던', 1),
 ('관람후', 1),
 ('비추!!!', 1),
 ('연기자는', 1),
 ('최악임', 1),
 ('무엇인지...', 1),
 ('결여되고', 1),
 ('한물', 1)]

In [51]:
# 단어 분포 확인.


In [52]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.charts.attributes import CatAttr
from bokeh.charts import Dot
# from bokeh.models import ColumnDataSource, LabelSet
# from bokeh.models import HoverTool
output_notebook() # 주피터 노트북에서 볼때,
#output_file("chart.html") # 주피터노트북이 아닌 html로 출력할때,

Loading BokehJS ...

In [53]:
data = {
    'token': [ ele[0] for ele in vocab][2:500],
    'freq': [ ele[1] for ele in vocab][2:500]
}
tools = "pan,wheel_zoom,box_zoom,reset,resize"

dots = Dot(data, values='freq', label=CatAttr(columns=['token'], sort=False), legend=False, title="Python Interpreter Sampling", width=600)

show(dots)

In [54]:
# 저빈도 단어는 "UNKNOWN_TOKEN"로 처리
unknown_token = "UNKNOWN_TOKEN"

In [55]:
def build_vocab(sentences, vocab_size, unknown_token):
    # vocab
    word_freq = collections.Counter(text_tokenized)
    print("전체 단어 개수 : ", len(word_freq.items()))
    vocab = word_freq.most_common(vocab_size - 1)
    
    # Mapping from index to word
    id2word = [x[0] for x in vocab]
    id2word.append(unknown_token) # 사전에 없는단어 처리.
    # Mapping from word to index
    word2id = {w: i for i, w in enumerate(id2word)}

    return (word2id, id2word)

In [56]:
word2id, id2word = build_vocab(text_tokenized, vocab_size, unknown_token)

전체 단어 개수 :  4724


In [57]:
word2id

{'지루했음': 651,
 '나간': 418,
 '단편': 652,
 '였습니다': 653,
 '캐릭터에': 654,
 '아까워': 182,
 '존나': 655,
 'ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ': 681,
 '만드니?': 657,
 '시간아깝다': 658,
 '하지않나': 659,
 'ㅓ': 308,
 '별로였다..내': 662,
 '좋다': 663,
 '9점이': 664,
 '1시간30분': 2736,
 '긴장감이': 666,
 '느낌.': 183,
 '이해못하는': 667,
 '늘어짐...': 668,
 '참아내느라': 669,
 '무지하게': 670,
 '20년': 671,
 '닮았다는건지...': 672,
 '스탈.': 673,
 '몇개있고...남자친구는': 674,
 '비어버린': 675,
 '짜내려고': 676,
 '지루의': 677,
 '재미없음ㅡㅡㅡㅡㅡㅋㅋ': 823,
 '꼰대가': 678,
 '여배우가': 679,
 '명작이었음~!': 680,
 '내용이라든가....': 656,
 '왕자를': 682,
 '꼭': 190,
 '생각하심??': 683,
 'UNKNOWN_TOKEN': 2999,
 '억지네': 684,
 '영': 685,
 'u': 686,
 '..;;': 687,
 '학생시절': 688,
 '성공기': 689,
 '전개도': 690,
 '만들었네요': 691,
 '특이할것': 692,
 '복수': 693,
 '여자판': 694,
 '왜보냐~': 695,
 '루이의': 696,
 '재미없다.': 184,
 '정말지루하고': 698,
 '적당.평점': 699,
 '하는건지': 700,
 '내': 58,
 '연락도씹어요!': 701,
 '않는다는것은': 998,
 '처음엔': 703,
 '문제임..1,2,3편까지가': 704,
 '안남': 705,
 '감독들이': 706,
 '기대': 101,
 '같지만...이런': 707,
 '돋는다루의': 709,
 '마피아비판도': 711,
 '상승이미지가': 712,
 '좋을': 713,
 '저

In [58]:
# 단어 -> index
word2id["영화"]

4

In [59]:
id2word[:10]

['SENTENCE_START',
 'SENTENCE_END',
 '너무',
 '진짜',
 '영화',
 '정말',
 '평점이',
 '그냥',
 '영화.',
 '다']

In [60]:
id2word[-10:]

['영화는...이라는',
 '새것이었던',
 '관람후',
 '비추!!!',
 '연기자는',
 '최악임',
 '무엇인지...',
 '결여되고',
 '한물',
 'UNKNOWN_TOKEN']

In [61]:
# 전체 단어 개수
len(id2word)

3000

In [62]:
# index -> 단어
id2word[2]

'너무'

In [63]:
# 원래 이랬던 리뷰 데이터가..
text_tokenized[:30]

['SENTENCE_START',
 '극장에서',
 '돈내고',
 '보긴아깝다',
 'SENTENCE_END',
 'SENTENCE_START',
 '진짜,,',
 'SENTENCE_END',
 'SENTENCE_START',
 '훌륭하지만',
 '비약적인',
 '스토리',
 'SENTENCE_END',
 'SENTENCE_START',
 'OOO야..',
 '아',
 '진짜',
 '후회했음',
 '후회막급',
 '!',
 '!',
 '오빠테',
 '욕먹었음...',
 '-_-',
 'SENTENCE_END',
 'SENTENCE_START',
 '잼씀',
 'SENTENCE_END',
 'SENTENCE_START',
 '지']

In [64]:
# 저빈도 단어 필터링 ( 토크나이징된 데이터 중 단어 사전에 등록된 것이 있으면 그대로 쓰고, 없으면 UNKNOWN_TOKEN로 대체.)
text_tokenized = [w if w in word2id else unknown_token for w in text_tokenized]
text_tokenized[:30]

['SENTENCE_START',
 '극장에서',
 '돈내고',
 'UNKNOWN_TOKEN',
 'SENTENCE_END',
 'SENTENCE_START',
 '진짜,,',
 'SENTENCE_END',
 'SENTENCE_START',
 '훌륭하지만',
 '비약적인',
 '스토리',
 'SENTENCE_END',
 'SENTENCE_START',
 'UNKNOWN_TOKEN',
 '아',
 '진짜',
 'UNKNOWN_TOKEN',
 'UNKNOWN_TOKEN',
 '!',
 '!',
 '오빠테',
 '욕먹었음...',
 '-_-',
 'SENTENCE_END',
 'SENTENCE_START',
 '잼씀',
 'SENTENCE_END',
 'SENTENCE_START',
 'UNKNOWN_TOKEN']

In [65]:
# 위의 데이터에서 각 단어를 학습을 위해 id로 치환.
tensor = np.array(list(map(word2id.get, text_tokenized)))

In [66]:
tensor[:100]

array([   0,   84,  429, 2999,    1,    0, 2230,    1,    0, 1580, 2012,
         43,    1,    0, 2999,   48,    3, 2999, 2999,  425,  425, 2734,
       1209,  102,    1,    0, 2918,    1,    0, 2999, 2999, 1938, 2999,
        689, 1385, 1017,  711, 1387, 2596, 2999,    9, 1978, 2668, 2641,
       2999, 2999,  796, 2030, 1276, 2683,  356, 2999, 1185,    1,    0,
       2999,    1,    0,    3, 2746,  234,    1,    0,  184,    1,    0,
       2487,  487, 2999,   27, 2999, 1444,  648,  511,    1,    0, 2740,
          1,    0,   65, 1102,  133,    1,    0, 2999,    1,    0, 2828,
        370,   10,   26, 2999,  197,  367,  216,  181,    4,    1,    0,
         19])

## 3. Build Batches for Training

In [35]:
batch_size = 20 # 배치사이즈
seq_length = 10 # rnn 모델에서 t의 사이즈

In [36]:
def create_batches(tensor, batch_size, seq_length):
    num_batches = int(tensor.size / (batch_size * seq_length))
    if num_batches == 0:
        assert False, "Not enough data. Make seq_length and batch_size small."

    tensor = tensor[:num_batches * batch_size * seq_length]
    xdata = tensor
    ydata = np.copy(tensor)

    ydata[:-1] = xdata[1:]
    ydata[-1] = xdata[0]
    x_batches = np.split(xdata.reshape(batch_size, -1), num_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), num_batches, 1)

    return x_batches, y_batches

In [37]:
x_batches, y_batches = create_batches(tensor, batch_size, seq_length)
num_batches = len(x_batches)

In [38]:
num_batches

43

### 전처리 결과

In [39]:
# 원 문장
text_tokenized[:5]

['SENTENCE_START', '극장에서', '돈내고', '보긴아깝다', 'SENTENCE_END']

In [40]:
# 원 문장 각 단어를 인덱스로 표현
[word2id[w] for w in text_tokenized[:5]]

[1, 94, 559, 2437, 0]

In [41]:
# 학습에 사용되는 X_train 데이터
x_batches[1]

array([[2999,   40,    0,    1, 2009,   53,    3, 2999, 1857,  529],
       [   0,    1, 2999,  191, 2999, 2999, 1885,    0,    1,  734],
       [   1, 1925, 1187,  145, 1585,    0,    1,  213, 2999, 2999],
       [ 158, 2425,    0,    1,  559, 2518, 2999,   43,    0,    1],
       [ 150, 1682,    0,    1, 2999,    5, 2999,  325, 2999, 2999],
       [   0,    1, 2785,  114, 1366,    0,    1,    2, 2364,   24],
       [ 551,  622,   77, 1285,  967, 2999,    0,    1,  226,    0],
       [   1, 1939,  706, 1798,    4, 2999, 2999, 2999,    2, 1483],
       [2999, 2999, 1458,    0,    1, 2999, 1686, 2764, 2642,   60],
       [   0,    1, 1491,    0,    1,   41, 2999,  505, 2603,   70],
       [1859,    0,    1,  412, 2999,  514,    6, 2999,    0,    1],
       [ 523,    9,  186,   14, 2616,  799,    0,    1, 2261, 2999],
       [2245,    4,  259,    0,    1,  129, 2999, 1412, 2999,    0],
       [2800,    0,    1,   52, 2999,  642, 2999, 1314, 2719,    0],
       [2862,  589, 2171, 2999,   

In [42]:
# 학습에 사용되는 y_train 데이터
y_batches[1]

array([[  40,    0,    1, 2009,   53,    3, 2999, 1857,  529,  529],
       [   1, 2999,  191, 2999, 2999, 1885,    0,    1,  734,   27],
       [1925, 1187,  145, 1585,    0,    1,  213, 2999, 2999,    0],
       [2425,    0,    1,  559, 2518, 2999,   43,    0,    1, 1259],
       [1682,    0,    1, 2999,    5, 2999,  325, 2999, 2999, 2999],
       [   1, 2785,  114, 1366,    0,    1,    2, 2364,   24,  230],
       [ 622,   77, 1285,  967, 2999,    0,    1,  226,    0,    1],
       [1939,  706, 1798,    4, 2999, 2999, 2999,    2, 1483, 2999],
       [2999, 1458,    0,    1, 2999, 1686, 2764, 2642,   60, 2999],
       [   1, 1491,    0,    1,   41, 2999,  505, 2603,   70, 1460],
       [   0,    1,  412, 2999,  514,    6, 2999,    0,    1,  503],
       [   9,  186,   14, 2616,  799,    0,    1, 2261, 2999,    0],
       [   4,  259,    0,    1,  129, 2999, 1412, 2999,    0,    1],
       [   0,    1,   52, 2999,  642, 2999, 1314, 2719,    0,    1],
       [ 589, 2171, 2999,    0,   

<br><br><br><br>
# Step 2. RNN Modeling
<hr style="border-color:#ff9900"> 
![](img/RNN_model.png)

## 1. Initialization
- 파라미터 U,V,W 를 초기화.

In [43]:
args = {}
args['data_dir'] = './data'
args['log_dir'] = './logs'
args['save_dir'] = './save'
args['rnn_size'] = 256
args['num_layers'] = 5
args['model'] = 'lstm'
args['batch_size'] = batch_size
args['seq_length'] = seq_length
args['num_epochs'] = 10
args['save_every'] = 1000
args['grad_clip'] = 5
args['learning_rate'] = 0.002 
args['decay_rate'] = 0.97 
args['vocab_size'] = vocab_size

<img src="https://qph.ec.quoracdn.net/main-qimg-1ec77cdbb354c3b9d439fbe436dc5d4f" />

In [44]:
with open(os.path.join(args['save_dir'], 'config.pkl'), 'wb') as f:
    pickle.dump(args, f)
with open(os.path.join(args['save_dir'], 'words_vocab.pkl'), 'wb') as f:
    pickle.dump((id2word, word2id), f)

In [45]:
class Model():
    def __init__(self, args, infer=False):
        self.args = args
        if infer:
            args['batch_size'] = 1
            args['seq_length'] = 1

        if args['model'] == 'rnn':
            cell_fn = rnn_cell.BasicRNNCell
        elif args['model'] == 'gru':
            cell_fn = rnn_cell.GRUCell
        elif args['model'] == 'lstm':
            cell_fn = rnn_cell.LSTMCell
        else:
            raise Exception("model type not supported: {}".format(args['model']))

        cell = cell_fn(args['rnn_size'])

        self.cell = cell = rnn_cell.MultiRNNCell([cell] * args['num_layers'])
        self.input_data = tf.placeholder(tf.int32, [args['batch_size'], args['seq_length']])
        self.targets = tf.placeholder(tf.int32, [args['batch_size'], args['seq_length']])
        self.initial_state = cell.zero_state(args['batch_size'], tf.float32)
        self.epoch_pointer = tf.Variable(0, name="epoch_pointer", trainable=False)
        self.batch_time = tf.Variable(0.0, name="batch_time", trainable=False)

        softmax_w = tf.get_variable("softmax_w", [args['rnn_size'], args['vocab_size']])
        softmax_b = tf.get_variable("softmax_b", [args['vocab_size']])
        with tf.device("/cpu:0"):
            embedding = tf.get_variable("embedding", [args['vocab_size'], args['rnn_size']])
            inputs = tf.split(1, args['seq_length'], tf.nn.embedding_lookup(embedding, self.input_data))
            inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

        def loop(prev, _):
            prev = tf.matmul(prev, softmax_w) + softmax_b
            prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
            return tf.nn.embedding_lookup(embedding, prev_symbol)

        outputs, last_state = seq2seq.rnn_decoder(inputs, self.initial_state, cell, loop_function=loop if infer else None, scope='rnnlm')
        output = tf.reshape(tf.concat(1, outputs), [-1, args['rnn_size']])
        self.logits = tf.matmul(output, softmax_w) + softmax_b
        self.probs = tf.nn.softmax(self.logits)
        loss = seq2seq.sequence_loss_by_example([self.logits],
                                                [tf.reshape(self.targets, [-1])],
                                                [tf.ones([args['batch_size'] * args['seq_length']])],
                                                args['vocab_size'])
        self.cost = tf.reduce_sum(loss) / args['batch_size'] / args['seq_length']
        self.final_state = last_state
        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), args['grad_clip'])
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def sample(self, sess, id2word, word2id, prime='SENTENCE_START'):
        state = sess.run(self.cell.zero_state(1, tf.float32))

        if not len(prime) or prime == " ":
            prime = random.choice(list(word2id.keys()))

        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return (int(np.searchsorted(t, np.random.rand(1) * s)))
        sent = [prime]
        word = prime.split()[-1]
        n = 0
        while sent[-1] != 'SENTENCE_END':
            x = np.zeros((1, 1))
            x[0, 0] = word2id[word]
            feed = {self.input_data: x, self.initial_state: state}
            probs, state = sess.run([self.probs, self.final_state], feed)
            p = probs[0]
            sample = weighted_pick(p)
            pred = id2word[sample]
            if pred == "UNKNOWN_TOKEN":
                continue
            sent.append(pred)
            word = pred
        return sent[1:-1]

In [46]:
model = Model(args)

In [47]:
with tf.Session() as sess:
    tf.global_variables_initializer().run() # tf 변수들 초기화
    saver = tf.train.Saver(tf.global_variables())

    for e in range(model.epoch_pointer.eval(), args['num_epochs']):
        print('\n\n--------- epoch : %i -----------\n\n' % e)
        sess.run(tf.assign(model.lr, args['learning_rate'] * (args['decay_rate'] ** e)))  # model.lr에 값 할당.
        state = sess.run(model.initial_state)
        speed = 0

        for b, (x_batch, y_batch) in enumerate(zip(x_batches, y_batches)):
            start = time.time()

            feed = {model.input_data: x_batch,
                    model.targets: y_batch,
                    model.initial_state: state,
                    model.batch_time: speed
                    }
            train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
            speed = time.time() - start
            if (e * num_batches + b) % args['batch_size'] == 0:
                print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}".format(e * num_batches + b, args['num_epochs'] * num_batches, e, train_loss, speed))
            if (e * num_batches + b) % args['save_every'] == 0 or (e == args['num_epochs'] - 1 and b == num_batches - 1):  # save for the last result
                checkpoint_path = os.path.join(args['save_dir'], 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=e * num_batches + b)
                print("model saved to {}".format(checkpoint_path))



--------- epoch : 0 -----------


0/430 (epoch 0), train_loss = 8.498, time/batch = 0.313
model saved to ./save/model.ckpt
20/430 (epoch 0), train_loss = 6.129, time/batch = 0.116
40/430 (epoch 0), train_loss = 6.346, time/batch = 0.109


--------- epoch : 1 -----------


60/430 (epoch 1), train_loss = 5.655, time/batch = 0.103
80/430 (epoch 1), train_loss = 6.462, time/batch = 0.106


--------- epoch : 2 -----------


100/430 (epoch 2), train_loss = 5.329, time/batch = 0.116
120/430 (epoch 2), train_loss = 5.828, time/batch = 0.114


--------- epoch : 3 -----------


140/430 (epoch 3), train_loss = 5.677, time/batch = 0.109
160/430 (epoch 3), train_loss = 5.567, time/batch = 0.112


--------- epoch : 4 -----------


180/430 (epoch 4), train_loss = 5.703, time/batch = 0.121
200/430 (epoch 4), train_loss = 5.848, time/batch = 0.116


--------- epoch : 5 -----------


220/430 (epoch 5), train_loss = 5.360, time/batch = 0.120
240/430 (epoch 5), train_loss = 5.633, time/batch = 0.116


-

<br><br><br><br>
# Step 3. Generating Text
<hr style="border-color:#ff9900"> 

### 저장된 모델 불러오기

In [48]:
args={}
args['save_dir'] = './save'
args['prime'] = 'SENTENCE_START'

In [49]:
with open(os.path.join(args['save_dir'], 'config.pkl'), 'rb') as f:
    saved_args = pickle.load(f)
with open(os.path.join(args['save_dir'], 'words_vocab.pkl'), 'rb') as f:
    id2word, word2id = pickle.load(f)

In [50]:
tf.reset_default_graph() # 그래프 리셋
model_gen = Model(saved_args, infer=True)

### 악평 생성하기

In [51]:
# 생성할 문장 수
num_sentences = 10
# 문장의 최소 길이
senten_min_length = 4

In [52]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state(args['save_dir'])
    print(ckpt.model_checkpoint_path)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        for _ in range(num_sentences):
            bot_sent = []
            while len(bot_sent) < senten_min_length:
                bot_sent = model_gen.sample(sess, id2word, word2id, args['prime'])
            print(' '.join(bot_sent))
            print()

./save/model.ckpt-429
머하는거지? 중간까지 손발 어설프다...지루하다.. 찬사를 너무 최저. 없음. 의리 가정에 죽겠다. 오와 없는 실화라는 할수 충분히 최악의 아쉽다 감동 재미없는

시리즈중 관계자들이 3류보다 서부극 의리!있는 작가가 SENTENCE_START 피해망상 SENTENCE_START SENTENCE_START 맞는건지는 괜찮았는데..이번작은 쓰기에 SENTENCE_START 허지웅을 SENTENCE_START 제대로된 영화...연출력도

발전한것 팬들 보고 SENTENCE_START 오바심함

매번 그들은 후회막급 영!! 낭비했다.

잃어버린 이해되질 없음..절대 이야기이네요...근디 감동포인트를 했다. 흠..그냥 실망 못한

역량이 일본만화는 떡 최악이라는 메릴스트립과 런닝타임만 쓰레기. SENTENCE_START 한 쌔고쌨는데ㅉㅉㅉ 한것 진짜 재밌었다... 아닌데... 또는 SENTENCE_START 진짜,, 재능 꺄악꺄악...-_- 추억의 인간의 애니를 수동적으로 너무 아까운영화 딱히 SENTENCE_START 오만은 SENTENCE_START SENTENCE_START 영화는 너무

ffff 몰락의 딱 박수칠때 지루하고 요새 SENTENCE_START 평론가들에게

학대해도 머하는거지? 미쳐서 시간아깝다. 울

실제로 2점부터 싫고쓰레기고엉터리고 안돼 ㅡㅡ 진짜 SENTENCE_START SENTENCE_START 이영화에 영화가

일탈도 아니라 7점 싫다 를 전작을 안됨 SENTENCE_START 보기도 SENTENCE_START SENTENCE_START 많았던것 못하네요.. 일본 듣기 쓰레기다. ㅋㅋ 잠들었다 뻔하고 체 SENTENCE_START 영화...연출력도 만들어도 8점대래? SENTENCE_START 영화관에 열심히 권력을 아.. 다 초,중,고딩용 SENTENCE_START 좋은 SENTENCE_START 이런 죽여야만

중간에 안됨 토나온다고 해달라.누가 별내용도 오지게 , 정신나간 하기는... 학생인 

<br><br><br><br>
# Evaluation & Challenging
<hr style="border-color:#ff9900"> 